In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 7.3 MB/s 
     |████████████████████████████████| 6.6 MB 57.4 MB/s 
     |████████████████████████████████| 101 kB 5.2 MB/s 
     |████████████████████████████████| 596 kB 48.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [11]:
from transformers import AutoTokenizer, AutoModelForPreTraining
import numpy as np
import torch

EMBEDDING_VECTOR_SHAPE = (768, 1)

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
model = AutoModelForPreTraining.from_pretrained("dbmdz/bert-base-turkish-uncased", output_hidden_states=True)

def getPhraseEmbedding(sentence, phrase):
    sentenceTokens = tokenizer.encode(sentence)  # Cümlenin BERT'e verilecek token id listesine dönüştürülmesi.

    wordEmbeddingList = model(torch.tensor([sentenceTokens]))[2][0][0]  # Modelin çalıştırılıp hidden state'inin (embedding'lerin elde edileceği yer) elde edilmesi.

    phraseTokens = tokenizer.encode(phrase)[1:-1]  # Embedding'i istenen phrase'in token id listesine dönüştürülmesi.

    ############################
    # Phrase start ve end indexlerinin belirlenmesi
    for sTokenIndex in range(len(sentenceTokens)):
        if sentenceTokens[sTokenIndex] == phraseTokens[0]:
            phraseStartIndex = sTokenIndex  #Inclusive
    for sTokenIndex in range(phraseStartIndex + 1, len(sentenceTokens)):
        phraseIndex = 1
        if phraseIndex >= len(phraseTokens) or sentenceTokens[sTokenIndex] != phraseTokens[phraseIndex]:
            phraseEndIndex = sTokenIndex  #Exclusive
        phraseIndex += 1
    ############################

    phraseMeanEmbedding = np.zeros(EMBEDDING_VECTOR_SHAPE)  # Phrase'deki her bir kelime için mean embedding'i tutacak olan vektör.
    for sentenceIndex in range(phraseStartIndex, phraseEndIndex):
        phraseMeanEmbedding = np.add(phraseMeanEmbedding, wordEmbeddingList[sentenceIndex].detach().numpy())
    phraseMeanEmbedding /= (phraseEndIndex - phraseStartIndex + 1)  # Phrase mean embedding elde edilmesi.
    
    return phraseMeanEmbedding[0]


sentence_piece_start = "Hava bugün "
sentence_piece_end = "."
word_1 = "berbat"
word_2 = "muhteşem"
print(np.dot(getPhraseEmbedding(sentence_piece_start + word_1 + sentence_piece_end, word_1), getPhraseEmbedding(sentence_piece_start + word_2 + sentence_piece_end, word_2)))

30.58405216649614
